In [1]:
import pandas as pd
from datasets import load_dataset
from torchtuning import koalpaca_dataset, kowiki_dataset, text_completion_dataset
from torchtune.models.llama3 import llama3_tokenizer

In [10]:
a = kowiki_dataset(data_files='data/20240520/kowikitext_20240520.csv', split='train')

In [3]:
b = text_completion_dataset(
    tokenizer=llama3_tokenizer,
    source='csv',
    column='text',
    max_seq_len=2048,
    data_files='data/20240520/kowikitext_20240520.csv',
    split='train'
)

In [4]:
b

In [2]:
df = pd.read_csv('data/20240520/kowikitext_20240520.csv')

In [3]:
df

,text
0,"= 플라톤 =\n\n플라톤(, 플라톤, ""넓은, 어깨 폭이 넓은"";Diogenes ..."
1,= 1280년 =\n\n= = 연호 = =\n\n원(元) 지원(至元) 17년\n일...
2,= 버니 차우 =\n\n버니 차우 또는 버니는 남아프리카 공화국의 빵 요리이다.\n...
3,= 분류:회네포스 출신 =
4,= 장솜이 =\n\n장솜이(1991년 2월 25일 ~ )는 대한민국의 배우이다.장솜...
...,...
1056993,= 후치다카역 =\n\n후치다카역은 일본 아이치현 아이사이시에 위치한 나고야 철도 ...
1056994,= You & Me (카드의 EP) =\n\nYou & Me는 혼성그룹 KARD의 ...
1056995,= 컨피규레이션 =\n\n컨피규레이션(configuration) 또는 컨피그(con...
1056996,= 위키백과:면책 조항 =\n\n__NOTOC__\n위키백과는 정확성을 보증하지 않...
